In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import os
import instructor
from rag.vector import ChromaRag

class Character(BaseModel):
    name: str
    description: int
    fact: List[str] = Field(..., description="A list of facts about the character")

class Answer(BaseModel):
    answer: str = Field(..., description="Exact requirement as per the user's query.")
    references: List[str] = Field(..., description="Key points or facts pulled from the documents. Include Metadata if available.")

client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  
    ),
    mode=instructor.Mode.JSON,
)

mdoel_embed = "mxbai-embed-large"
model_chat = "llama3.1:8b"
db_path = "markdown_db"
source_dir = r"C:\Users\shive\OneDrive\Documents\DnD\Adventures\Stone-Heart Hollow"
top_k = 5

collection_name = os.path.basename(source_dir)
chroma_rag = ChromaRag(
        source_directory=source_dir,
        collection_name=collection_name,
        db_path=db_path,
        model_name=mdoel_embed
    )

question = "What was in the letter sent to Councillor Bren?"
docs, ids, metadata = chroma_rag.retrieve(question, k=top_k)
docs_text = "\n---\n".join(docs)
# print(docs)
prompt = f"""
    ==================
    You are a virtual assistant for a Dungeon Master running Dungeons & Dragons (D&D) sessions.

    - Helping prepare and run sessions by providing ideas, rules clarifications, encounter design, and narrative suggestions.

    - Answering questions using provided campaign documents, focusing only on the relevant content.

    - If no documents are provided, rely on your general D&D knowledge (primarily 5th Edition unless specified).

    Be accurate, concise, and helpful. Prioritize clarity when referencing documents. Prioritize clarity and creativity when assisting with gameplay and storytelling.
    ==================

    ==================
    DOCUMENT CONTEXT:
    {docs_text}
    ==================

    ==================
    Query:
    {question}
    ==================
"""

resp = client.chat.completions.create(
    model="gemma3",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    response_model=Answer,
)
print(resp.model_dump_json(indent=2))
